In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

path = "C:/Users/agaci/OneDrive/Dokumenty/CogSci/VR/data_indication"

data = pd.read_json(path + "/2020_9_19_14_55_responses.json")

### Splitting data into four arrays

Front, back, left and right conditions

In [112]:
df_front = data[data.SENTENCE.isin(["przed"])]

df_back = data[data.SENTENCE.isin(["za"])]

df_right = data[data.SENTENCE.isin(["prawo"])]

df_left = data[data.SENTENCE.isin(["prawo"])]

df_front.head()

,F10_ANGLE,F10_COL,F10_DIST,F10_SCALE,F10_TYPE,F11_ANGLE,F11_COL,F11_DIST,F11_SCALE,F11_TYPE,...,gender,handedness,lang_prim,lang_sec,native,participant,response,rid,rt,selections
0,195,blue,90,medium,sphere,255,red,30,small,cube,...,M,R,PL,N,T,21,LEFT,KK,12752653,"[{'angle': 105, 'color': 'green', 'distance': ..."
8,315,blue,30,large,sphere,210,green,90,small,cube,...,M,R,PL,N,T,21,RIGHT,KK,8700660,"[{'angle': 75, 'color': 'green', 'distance': 3..."
13,225,blue,30,medium,cone,255,red,60,medium,cube,...,M,R,PL,N,T,21,RIGHT,KK,13851144,"[{'angle': 90, 'color': 'yellow', 'distance': ..."
17,345,blue,90,medium,sphere,255,yellow,30,medium,sphere,...,M,R,PL,N,T,21,LEFT,KK,11338253,"[{'angle': 165, 'color': 'yellow', 'distance':..."
25,300,red,90,large,cone,195,yellow,30,large,cylinder,...,M,R,PL,N,T,21,LEFT,KK,10238181,"[{'angle': 180, 'color': 'blue', 'distance': 9..."


### Reposition the figures

F1 is the centre-piece, needs to be set at a 90-degree angle.
The rest of the figures need to be shifted accordingly.

In [113]:
def reposition_figures(data_frame):

    for i in data_frame.index:

        if data_frame.at[i,"F1_ANGLE"] != 90:

            drift = data_frame.at[i,"F1_ANGLE"] - 90

            #reposition angles of figures

            for j in range(1, 13):

                col = "F" + str(j) + "_ANGLE"

                if data_frame.at[i, col]- drift < 0:

                    data_frame.at[i, col] = 360 - (drift - data_frame.at[i, col])

                else:

                    data_frame.at[i, col] -= drift

            #reposition angles of selected figures

            for k in range(len(data_frame.at[i, 'selections'])):

                if data_frame.at[i, 'selections'][k]['angle']- drift < 0:

                    data_frame.at[i, 'selections'][k]['angle'] = \
                        360 - (drift - data_frame.at[i, 'selections'][k]['angle'])

                else:

                    data_frame.at[i, 'selections'][k]['angle'] -= drift

    return data_frame

df = reposition_figures(df_left)

### For each item get locations of chosen figures


In [114]:
def get_selected_figures(data_frame):

    sol = []

    for i in data_frame.index:

        selections = data_frame.loc[i, "selections"]

        item_sol = []

        for elt in selections:

            item_sol.append( (elt['angle'], elt['distance']) )

        sol.append(item_sol)

    return sol

selected_figs = get_selected_figures(df)

### Create positions of all figures

In [115]:
def get_figure_locations(data_frame):

    sol = []

    k = -1

    for i in data_frame.index:

        k+= 1

        sub_sol = []

        item_angles = data_frame.loc[i, data_frame.columns.str.contains("ANGLE")]

        item_distances = data_frame.loc[i, data_frame.columns.str.contains("DIST")]

        for j in range(len(item_angles)):

            loc = (item_angles.values[j], item_distances.values[j])

            sub_sol.append(loc)

        sol.append(sub_sol)

    return sol


all_figs_positions = get_figure_locations(df)

### Decide which figures for all items were selected

In [116]:
def is_selected(selected_figures, all_figures):

    sol = []

    for row in range(len(all_figures)):

        item_sol = []

        for column in range(len(all_figures[row])):

            item_sol.append(all_figures[row][column] in selected_figures[row])

        sol.append(item_sol)

    return sol

selected = is_selected(selected_figs, all_figs_positions)


df_decisions = pd.DataFrame(selected)

### Create an array of angles/distances for plotting

In [117]:
def get_angles_or_distances(df, col_name):

    sol = df.loc[:, df.columns.str.contains(col_name)]

    sol.reset_index(inplace=True, drop=True)

    if col_name == "DIST":

        sol = sol/100

    if col_name == "ANGLE":

        sol = sol.values/ (180/np.pi)

    return sol

angles = get_angles_or_distances(df, "ANGLE")
distances = get_angles_or_distances(df, "DIST")

In [118]:
def color_points(df):

    df.replace(True, "Blue", inplace=True)
    df.replace(False, "Red", inplace=True)
    df.iloc[:, 3] = "Black"

    return 1

In [120]:
df_decisions

,0,1,2,3,4,5,6,7,8,9,10,11
0,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
1,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
2,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
3,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
4,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
5,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
6,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
7,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
8,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red
9,Red,Red,Red,Black,Blue,Blue,Blue,Blue,Blue,Red,Red,Red


In [122]:
color_points(df_decisions)

def plot_solution_pp(angles, distances, colors):

    for i in range(3,10,3):

        ang = angles[distances.F1_DIST == i/10]

        dst = distances[distances.F1_DIST == i/10]

        cols = colors[distances.F1_DIST == i/10]

        lst_col = []

        for i in cols.index:

            for j in cols:

                lst_col.append(cols.iloc[i, j])

        fig = plt.figure()
        ax = fig.add_subplot(projection='polar')
        ax.set_rticks([.3, .6, .9, 1.2])
        ax.set_ylim([0,1.2])
        c = ax.scatter(ang, dst, c = cols, alpha = 0.5)

    return 1

#plot_solution_pp(angles, distances, df_decisions)

     0    1    2      3     4     5     6     7     8    9    10   11
0   Red  Red  Red  Black  Blue  Blue  Blue  Blue  Blue  Red  Red  Red
1   Red  Red  Red  Black  Blue  Blue  Blue  Blue  Blue  Red  Red  Red
3   Red  Red  Red  Black  Blue  Blue  Blue  Blue  Blue  Red  Red  Red
10  Red  Red  Red  Black  Blue  Blue  Blue  Blue  Blue  Red  Red  Red
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11


IndexError: single positional indexer is out-of-bounds

In [ ]:
def run_all_for_pp(df):

    df = reposition_figures(df)

    selected_figs = get_selected_figures(df)

    all_figs_positions = get_figure_locations(df)

    selected = is_selected(selected_figs, all_figs_positions)

    df_decisions = pd.DataFrame(selected)

    color_points(df_decisions)

    angles = get_angles_or_distances(df, "ANGLE")

    distances = get_angles_or_distances(df, "DIST")

    plot_solution_pp(angles, distances, df_decisions)

    return 1


#run_all(df_back)

In [ ]:
import os

ALL_ANGLES = []

ALL_DISTANCES = []

ALL_COLOURS = []

def run_all_participants(path, condition):

    for filename in os.listdir(path):

        data = pd.read_json(path + "/" + filename)

        df = data[data.SENTENCE.isin([condition])]

        df = reposition_figures(df)

        selected_figs = get_selected_figures(df)

        all_figs_positions = get_figure_locations(df)

        selected = is_selected(selected_figs, all_figs_positions)

        df_decisions = pd.DataFrame(selected)

        color_points(df_decisions)

        ALL_ANGLES.append(get_angles_or_distances(df, "ANGLE"))

        ALL_DISTANCES.append(get_angles_or_distances(df, "DIST"))

        ALL_COLOURS.append(df_decisions)

    return 1

In [ ]:
def plot_solution_ALL(angles, distances, colors, condition):

    for i in range(3,10,3):

        ang = angles[distances.F1_DIST == i/10]

        dst = distances[distances.F1_DIST == i/10]

        cols = colors.iloc[distances[distances.F1_DIST == i/10].index]

        lst_col = []

        for row in cols.index:

            for val in cols.iloc[i, ]:

                lst_col.append(val)

        fig = plt.figure()
        ax = fig.add_subplot(projection='polar')
        ax.set_rticks([.3, .6, .9, 1.2])
        ax.set_ylim([0,1.2])
        c = ax.scatter(ang, dst, c = lst_col, alpha = 0.5)

        plt.savefig("C:/Users/agaci/OneDrive/Dokumenty/CogSci/VR/" + condition  + "." + str(i) + '.png')

    return 1

In [ ]:
condition = "prawo"

In [ ]:
run_all_participants(path, condition)

header = ALL_DISTANCES[0].columns

sol = ALL_DISTANCES[0]

for i in range(1,len(ALL_DISTANCES)):

    sol = np.append(sol, ALL_DISTANCES[i], axis=0)

sol = pd.DataFrame(sol, columns=header).reset_index(drop=True)

sol2 = ALL_ANGLES[0]

for i in range(1,len(ALL_ANGLES)):

    sol2 = np.append(sol2, ALL_ANGLES[i], axis=0)

sol2 = pd.DataFrame(sol2).reset_index(drop=True)

sol2

sol3 = ALL_COLOURS[0]

for i in range(1, len(ALL_COLOURS)):

    sol3 = pd.concat([sol3, ALL_COLOURS[i]])

sol3.reset_index(drop=True)

plot_solution_ALL(sol2, sol, sol3, condition)